In [1]:
import pickle

In [2]:
train = pickle.load(open('/tmp/citi/train_sample.pckl/train_sample.pckl', 'rb'))

In [3]:
train['lat'] = train['location'].map(lambda x: x['latitude'])
train['lon'] = train['location'].map(lambda x: x['longitude'])
train['is_verygood'] = train['description'].map(lambda x: 'отличн'  in x.lower() or \
                                                          'perfect' in  x.lower() or \
                                                          'идеал'   in  x.lower())

train['is_free'] = train['description'].map(lambda x: 'бесплатно' in x.lower() or \
                                                      'даром' in x.lower() or \
                                                      'отдам за киндер' in x.lower())

In [4]:
data = train[['category', 'subcategory', 'price', 'lat', 'lon','is_verygood','is_free']]

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('price', axis=1),
    data['price'],
    test_size=0.33, random_state=42)

In [7]:
from sklearn.ensemble import RandomForestRegressor

In [10]:
rf = RandomForestRegressor(n_estimators=1000, n_jobs=-1)

In [11]:
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=-1, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [12]:
from sklearn.metrics import mean_squared_log_error

In [13]:
mean_squared_log_error(y_test, rf.predict(X_test))**0.5

2.453224288168561